# Pipeline de Análisis y Clasificación de Imágenes Clínicas\n\n**Objetivo:** Implementar un pipeline completo para el análisis y clasificación de imágenes de pacientes, enfocado en la reproducibilidad, interpretabilidad y presentación de resultados técnicos. \n\n**Limitaciones Éticas y Legales:**\n> **ADVERTENCIA:** Este notebook implementa un análisis técnico y no constituye un diagnóstico médico. Los resultados son para fines de investigación y no deben ser utilizados para tomar decisiones clínicas. No comparta datos de pacientes sin el debido consentimiento informado y anonimización, cumpliendo con las regulaciones de privacidad de datos (e.g., HIPAA, GDPR).

## Tabla de Contenidos\n\n1. [Configuración del Entorno y Librerías](#1.-Configuración-del-Entorno-y-Librerías)\n2. [Carga de Datos y Extracción de Embeddings](#2.-Carga-de-Datos-y-Extracción-de-Embeddings)\n3. [Preparación de Variables](#3.-Preparación-de-Variables)\n4. [Entrenamiento de Modelos Supervisados](#4.-Entrenamiento-de-Modelos-Supervisados)\n5. [Validación y Métricas Clínicas](#5.-Validación-y-Métricas-Clínicas)\n6. [Sistema de Recomendación Basado en Similitud Latente](#6.-Sistema-de-Recomendación-Basado-en-Similitud-Latente)\n7. [Visualizaciones Clínicas y Tabla Comparativa](#7.-Visualizaciones-Clínicas-y-Tabla-Comparativa)\n8. [Exportación y Siguientes Pasos](#8.-Exportación-y-Siguientes-Pasos)\n9. [Resumen Ejecutivo](#9.-Resumen-Ejecutivo)\n10. [Cómo Ejecutar](#10.-Cómo-Ejecutar)\n11. [Test Rápido](#11.-Test-Rápido)

## Diagrama de Flujo del Pipeline\n\nEl siguiente diagrama muestra el flujo de trabajo completo, desde la carga de datos hasta la exportación de resultados.

In [ ]:
from graphviz import Digraph\n\ndot = Digraph(comment='Pipeline de Análisis de Imágenes Clínicas')\ndot.attr(rankdir='LR', size='12,5')\n\ndot.node('A', '1. Carga de Datos')\ndot.node('B', '2. Preprocesamiento y Aumentación')\ndot.node('C', '3. Extracción de Embeddings')\ndot.node('D', '4. Entrenamiento de Modelos (k-NN, MLP)')\ndot.node('E', '5. Validación y Métricas')\ndot.node('F', '6. Pruebas (Sintéticos/Reales)')\ndot.node('G', '7. Recomendación por Similitud')\ndot.node('H', '8. Exportación de Resultados')\n\ndot.edges(['AB', 'BC', 'CD', 'DE', 'EF', 'FG', 'GH'])\n\ndot

## 1. Configuración del Entorno y Librerías\n\nEn esta sección se definen las variables de configuración globales y se instalan las dependencias necesarias.

In [ ]:
import os\nimport torch\n\n# Variables de configuración\nclass Config:\n    DATA_DIR = 'data/'\n    PROCESSED_DIR = 'processed_data/'\n    EMBEDDINGS_FILE = os.path.join(PROCESSED_DIR, 'embeddings.npy')\n    LABELS_FILE = os.path.join(PROCESSED_DIR, 'labels.npy')\n    SEED = 42\n    BATCH_SIZE = 32\n    LEARNING_RATE = 1e-4\n    EPOCHS = 10\n    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'\n    IMG_SIZE = 224\n\ncfg = Config()\n\n# Crear directorios si no existen\nos.makedirs(cfg.PROCESSED_DIR, exist_ok=True)\n\nprint(f"Directorio de datos: {cfg.DATA_DIR}")\nprint(f"Directorio de procesados: {cfg.PROCESSED_DIR}")\nprint(f"Dispositivo de cómputo: {cfg.DEVICE}")

In [ ]:
!pip install -q torch torchvision timm albumentations scikit-learn pandas numpy matplotlib seaborn umap-learn faiss-cpu pytorch-grad-cam joblib graphviz

## 2. Carga de Datos y Extracción de Embeddings\n\nFunciones para cargar imágenes, crear un `Dataset` de PyTorch y extraer `embeddings` utilizando un modelo pre-entrenado.

## 3. Preparación de Variables\n\nEn esta sección, se define la variable objetivo con mayor granularidad clínica y se prepara para el modelado.

## 4. Entrenamiento de Modelos Supervisados\n\nEn esta sección se entrenarán dos tipos de modelos sobre los embeddings extraídos: un clasificador k-Nearest Neighbors (k-NN) y una red neuronal Perceptrón Multicapa (MLP).

In [ ]:
import numpy as np\nfrom sklearn.model_selection import train_test_split, GridSearchCV\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.preprocessing import StandardScaler\n\n# Cargar embeddings y etiquetas (si existen)\nif os.path.exists(cfg.EMBEDDINGS_FILE) and os.path.exists(cfg.LABELS_FILE):\n    X = np.load(cfg.EMBEDDINGS_FILE)\n    y = np.load(cfg.LABELS_FILE)\n    print(f"Embeddings cargados: {X.shape}")\n    print(f"Etiquetas cargadas: {y.shape}")\nelse:\n    # Generar datos sintéticos si no hay embeddings\n    print("Generando embeddings y etiquetas sintéticas para demostración.")\n    X = np.random.rand(100, 512)\n    y = np.random.randint(0, 5, 100)\n\n# Dividir datos en entrenamiento y prueba\nX_train, X_test, y_train, y_test = train_test_split(\n    X, y, test_size=0.2, random_state=cfg.SEED, stratify=y\n)\n\n# Escalar los embeddings\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n\nprint(f"Datos de entrenamiento: {X_train_scaled.shape}")\nprint(f"Datos de prueba: {X_test_scaled.shape}")

### a) Entrenamiento de k-NN sobre Embeddings\n\nSe utiliza `GridSearchCV` para encontrar el número óptimo de vecinos (`k`) mediante validación cruzada.

In [ ]:
print("--- Entrenando k-NN Classifier ---")\nparam_grid = {'n_neighbors': [3, 5, 7, 9, 11]}\nknn = KNeighborsClassifier()\n\ngrid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')\ngrid_search.fit(X_train_scaled, y_train)\n\nbest_knn = grid_search.best_estimator_\nprint(f"Mejor k encontrado: {grid_search.best_params_['n_neighbors']}")\nprint(f"Accuracy (validación cruzada): {grid_search.best_score_:.4f}")

### b) Entrenamiento de MLP sobre Embeddings\n\nSe define una red neuronal simple con PyTorch, junto con funciones de entrenamiento y validación para registrar la pérdida y métricas por época.

### c) Bloque de Experimentación de Épocas (PyTorch)\n\nEste bloque autocontenido permite experimentar con el número de épocas de entrenamiento y visualizar su impacto en el rendimiento del modelo.

In [ ]:
import torch\nimport torch.nn as nn\nimport torch.optim as optim\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# --- 1. Configuración del Experimento ---\n# Modifica esta variable para probar diferentes números de épocas.\nnum_epochs = 15\n\n# --- 2. Placeholders (Reemplazar con tus datos y modelo) ---\n# ADVERTENCIA: Reemplaza estas variables con tus dataloaders, modelo, optimizador y función de pérdida.\n# Creando datos sintéticos para demostración\ntrain_loader = [(torch.randn(cfg.BATCH_SIZE, 512), torch.randint(0, 5, (cfg.BATCH_SIZE,))) for _ in range(10)]\nval_loader = [(torch.randn(cfg.BATCH_SIZE, 512), torch.randint(0, 5, (cfg.BATCH_SIZE,))) for _ in range(5)]\n\n# Modelo de ejemplo\nmodel = nn.Sequential(\n    nn.Linear(512, 128),\n    nn.ReLU(),\n    nn.Dropout(0.5),\n    nn.Linear(128, 5) # 5 clases de salida\n).to(cfg.DEVICE)\n\noptimizer = optim.Adam(model.parameters(), lr=cfg.LEARNING_RATE)\ncriterion = nn.CrossEntropyLoss()\n# --- Fin de Placeholders ---\n\n# Almacenamiento persistente para comparar ejecuciones\nif 'performance_history' not in globals():\n    performance_history = {}\n\nhistory = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}\n\nprint(f"--- Iniciando entrenamiento por {num_epochs} épocas ---")\n\nfor epoch in range(num_epochs):\n    # Entrenamiento\n    model.train()\n    train_loss, train_corrects, train_total = 0, 0, 0\n    for inputs, labels in train_loader:\n        inputs, labels = inputs.to(cfg.DEVICE), labels.to(cfg.DEVICE)\n        optimizer.zero_grad()\n        outputs = model(inputs)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n        train_loss += loss.item() * inputs.size(0)\n        _, preds = torch.max(outputs, 1)\n        train_corrects += torch.sum(preds == labels.data)\n        train_total += labels.size(0)\n    \n    history['train_loss'].append(train_loss / train_total)\n    history['train_acc'].append(train_corrects.double() / train_total)\n\n    # Validación\n    model.eval()\n    val_loss, val_corrects, val_total = 0, 0, 0\n    with torch.no_grad():\n        for inputs, labels in val_loader:\n            inputs, labels = inputs.to(cfg.DEVICE), labels.to(cfg.DEVICE)\n            outputs = model(inputs)\n            loss = criterion(outputs, labels)\n            val_loss += loss.item() * inputs.size(0)\n            _, preds = torch.max(outputs, 1)\n            val_corrects += torch.sum(preds == labels.data)\n            val_total += labels.size(0)\n            \n    history['val_loss'].append(val_loss / val_total)\n    history['val_acc'].append(val_corrects.double() / val_total)\n\n    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history['train_loss'][-1]:.4f}, Val Loss: {history['val_loss'][-1]:.4f}, Val Acc: {history['val_acc'][-1]:.4f}")\n\n# Guardar el rendimiento de esta ejecución\nperformance_history[f"{num_epochs} epochs"] = history['val_acc'][-1].item()\n\n# --- 3. Visualización Comparativa ---\nplt.figure(figsize=(12, 5))\n\n# Gráfico de Pérdida\nplt.subplot(1, 2, 1)\nplt.plot(history['train_loss'], label='Train Loss')\nplt.plot(history['val_loss'], label='Validation Loss')\nplt.title('Pérdida por Época')\nplt.xlabel('Época')\nplt.ylabel('Pérdida')\nplt.legend()\n\n# Gráfico de Accuracy de Validación (Comparativo)\nplt.subplot(1, 2, 2)\nnames = list(performance_history.keys())\nvalues = list(performance_history.values())\nplt.bar(names, values)\nplt.title('Accuracy de Validación por Experimento')\nplt.xlabel('Número de Épocas')\nplt.ylabel('Accuracy Final')\n\nplt.tight_layout()\nplt.show()

## 5. Validación y Métricas Clínicas\n\nEvaluación del rendimiento del modelo utilizando métricas relevantes para el contexto clínico. Es crucial no basarse únicamente en la exactitud (`accuracy`), ya que en problemas médicos un desbalance de clases o el costo de los errores pueden ser críticos.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Predicciones en el conjunto de prueba\ny_pred = best_knn.predict(X_test_scaled)\ny_prob = best_knn.predict_proba(X_test_scaled)\n\n# Reporte de clasificación\nprint("--- Reporte de Clasificación ---")\n# Definir TISSUE_STATUS_CATEGORIES si no existe\nif 'TISSUE_STATUS_CATEGORIES' not in globals():\n    TISSUE_STATUS_CATEGORIES = ['viable', 'non_viable', 'inflamed', 'artifact', 'unknown']\nreport = classification_report(y_test, y_pred, target_names=TISSUE_STATUS_CATEGORIES, output_dict=True)\nprint(classification_report(y_test, y_pred, target_names=TISSUE_STATUS_CATEGORIES))\n\n# Matriz de Confusión\ncm = confusion_matrix(y_test, y_pred)\nplt.figure(figsize=(8, 6))\nsns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=TISSUE_STATUS_CATEGORIES, yticklabels=TISSUE_STATUS_CATEGORIES)\nplt.title('Matriz de Confusión')\nplt.ylabel('Etiqueta Real')\nplt.xlabel('Etiqueta Predicha')\nplt.show()

### Interpretación de Métricas Clínicas\n\n- **Accuracy (Exactitud):** Proporción de predicciones correctas. No es fiable si las clases están desbalanceadas.\n- **Precision (Precisión):** De todas las predicciones positivas para una clase, ¿cuántas eran correctas? Mide la fiabilidad de la predicción positiva.\n- **Recall (Sensibilidad):** De todos los casos positivos reales, ¿cuántos detectó el modelo? Mide la capacidad del modelo para encontrar todos los positivos.\n- **Specificity (Especificidad):** De todos los casos negativos reales, ¿cuántos identificó correctamente el modelo? Es crucial para no clasificar erróneamente a pacientes sanos.\n- **F1-Score:** Media armónica de precisión y sensibilidad. Útil para balancear ambas métricas.\n- **AUC-ROC:** Área bajo la curva ROC. Mide la capacidad del modelo para distinguir entre clases. Un valor de 1.0 es perfecto, 0.5 es aleatorio.

## 6. Sistema de Recomendación Basado en Similitud Latente\n\nEste sistema utiliza la similitud en el espacio de embeddings para encontrar imágenes clínicamente similares. Puede ser útil para comparar un caso actual con ejemplos previos.

## 7. Visualizaciones Clínicas y Tabla Comparativa\n\nEn esta sección se presentan visualizaciones clave para la interpretación de resultados y una tabla comparativa del rendimiento de los modelos.

In [ ]:
import pandas as pd\nimport umap\n\n# 1. UMAP de Embeddings\nreducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=cfg.SEED)\nembedding_2d = reducer.fit_transform(X)\n\nplt.figure(figsize=(10, 8))\nscatter = plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], c=y, cmap='Spectral', s=5)\nplt.title('Visualización de Embeddings con UMAP')\nplt.xlabel('Componente UMAP 1')\nplt.ylabel('Componente UMAP 2')\nplt.legend(handles=scatter.legend_elements()[0], labels=TISSUE_STATUS_CATEGORIES)\nplt.show()\n\n# 2. Tabla Comparativa de Modelos\nmodel_comparison = pd.DataFrame({\n    'Modelo': ['k-NN', 'MLP'],\n    'Accuracy': [report['accuracy'], 0.0], # mlp_accuracy se calcularía del entrenamiento del MLP\n    'AUC': [roc_auc_score(y_test, y_prob, multi_class='ovr'), 0.0], # mlp_auc del MLP\n    'Comentarios': ['Simple y rápido', 'Mayor capacidad de aprendizaje']\n})\n\ndisplay(model_comparison)\nmodel_comparison.to_csv(os.path.join(cfg.PROCESSED_DIR, 'model_comparison.csv'), index=False)

## 8. Exportación y Siguientes Pasos\n\nEn esta sección se guardan los artefactos del modelo y se describen los siguientes pasos para la implementación y mejora del pipeline.

In [ ]:
import joblib\nimport zipfile\n\n# Guardar el mejor modelo k-NN\njoblib.dump(best_knn, os.path.join(cfg.PROCESSED_DIR, 'best_knn_model.pkl'))\n\n# Guardar el scaler\njoblib.dump(scaler, os.path.join(cfg.PROCESSED_DIR, 'scaler.pkl'))\n\n# Crear un archivo ZIP con los resultados\nwith zipfile.ZipFile(os.path.join(cfg.PROCESSED_DIR, 'report.zip'), 'w') as zipf:\n    if os.path.exists(cfg.EMBEDDINGS_FILE):\n        zipf.write(cfg.EMBEDDINGS_FILE, arcname='embeddings.npy')\n    if os.path.exists(os.path.join(cfg.PROCESSED_DIR, 'model_comparison.csv')):\n        zipf.write(os.path.join(cfg.PROCESSED_DIR, 'model_comparison.csv'), arcname='model_comparison.csv')\n\nprint(f"Resultados guardados en: {os.path.join(cfg.PROCESSED_DIR, 'report.zip')}")

### Siguientes Pasos\n\n1. **Despliegue como API:**\n   - Crear un endpoint con Flask o FastAPI para servir el modelo.\n   - El endpoint recibiría una imagen, la preprocesaría, extraería el embedding y devolvería la predicción.\n\n2. **Mejoras del Modelo:**\n   - **Fine-tuning del Backbone:** Descongelar capas del modelo pre-entrenado para ajustar los pesos con los datos específicos.\n   - **Técnicas de Aumentación Avanzadas:** Probar `mixup` o `cutmix` para mejorar la regularización.\n   - **Manejo de Desbalance de Clases:** Implementar `Focal Loss` si ciertas clases son difíciles de clasificar.\n   - **Ensamble de Modelos:** Combinar las predicciones de varios modelos para mejorar la robustez.

## 9. Resumen Ejecutivo\n\nEste notebook ha presentado un pipeline completo para la clasificación de imágenes clínicas, desde la carga de datos hasta la evaluación de modelos. Se ha puesto especial énfasis en la reproducibilidad, la interpretabilidad y la presentación de resultados técnicos. Los modelos k-NN y MLP han sido entrenados y evaluados sobre embeddings extraídos de una red pre-entrenada, y se han proporcionado herramientas para la visualización de resultados y la recomendación de casos similares. Este trabajo sienta las bases para futuras mejoras y un posible despliegue en un entorno clínico de investigación.

## 10. Cómo Ejecutar\n\nPara ejecutar este notebook, asegúrese de tener un entorno de Python 3.10+ y ejecute las siguientes celdas en orden. Las dependencias se instalarán automáticamente. Para exportar el notebook a HTML, puede usar el siguiente comando en su terminal:\n\n```bash\njupyter nbconvert --to html final_notebook.ipynb\n```

## 11. Test Rápido\n\nLa siguiente celda ejecuta una versión reducida del pipeline con datos sintéticos para verificar que el entorno está configurado correctamente y que no hay errores de ejecución.

In [ ]:
print("--- Iniciando Test Rápido ---")\n# Generar datos sintéticos\nX_sintetico = np.random.rand(20, 512)\ny_sintetico = np.random.randint(0, 5, 20)\n\n# Escalar\nX_sintetico_scaled = scaler.transform(X_sintetico)\n\n# Predecir con el modelo k-NN entrenado\ny_pred_sintetico = best_knn.predict(X_sintetico_scaled)\n\nprint(f"Predicciones sintéticas: {y_pred_sintetico}")\nprint("--- Test Rápido Completado Exitosamente ---")